<a href="https://colab.research.google.com/github/Dipesh-empire/customer-churn-prediction/blob/main/Telco_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer.csv to customer.csv


# 1) Logisitic Regression Model


In [ ]:
# For logistic regression model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
df = pd.read_csv('customer.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
#Changing the datatype of total charges to float
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors= 'coerce')

In [ ]:
df.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
Partner,0
Dependents,0
tenure,0
PhoneService,0
MultipleLines,0
InternetService,0
OnlineSecurity,0


In [ ]:
# it is safe and best option to drop those 11 missing rows

df.dropna(inplace=True)

In [ ]:
#Converting(encoding) churn column into 0 and 1 for ML model
df['Churn']= df['Churn'].map({'No':0,'Yes':1})

In [ ]:
#dropping column that is not necessary for prediction
df.drop(['customerID'], axis=1, inplace=True)

In [ ]:
#Using one hot encoding
df= pd.get_dummies(df, drop_first=True)

In [ ]:
#Normalizing the column
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,-1.280248,-1.161694,-0.994194,0,False,True,False,False,True,...,False,False,False,False,False,False,True,False,True,False
1,0,0.064303,-0.260878,-0.173740,0,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
2,0,-1.239504,-0.363923,-0.959649,1,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
3,0,0.512486,-0.747850,-0.195248,0,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
4,0,-1.239504,0.196178,-0.940457,1,False,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False


In [ ]:
#Giving features x and y different features
#Values in x is beside the churn which will act as input
x = df.drop('Churn', axis=1)
#Values in y is churn value which will act as output
y = df['Churn']

# Train-test split helps us to train 80% data and test 20% data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
#Predict on test set
y_pred= model.predict(X_test)

#Print evaluation metrics
print("Confusion matrix:")
print(confusion_matrix(y_test,y_pred))

print("\n classification report :")
print(classification_report(y_test,y_pred))

Confusion matrix:
[[915 118]
 [182 192]]

 classification report :
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.62      0.51      0.56       374

    accuracy                           0.79      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407



##2) Transformer model for tabular data

In [ ]:
#For transformer model for tabular data
#Importing pytorch libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
#This class helps us to use our Pandas DataFrames with PyTorch’s DataLoader.
class ChurnDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:

# Convert all feature columns to float
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Convert target to float as well
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)


In [ ]:
#Batch the data into groups (e.g., 64 rows per batch), and shuffle the training data.

train_ds = ChurnDataset(X_train, y_train)
test_ds = ChurnDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)


In [ ]:
class TabularTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()

        # Embed tabular data into a higher-dimensional space
        self.embedding = nn.Linear(input_dim, d_model)

        # Define Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Final layers to convert attention output into prediction
        self.head = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),             #Converting our negative into 0 and positive being as it is
            nn.Linear(32, 1),
            nn.Sigmoid()  # Since output is a probability (binary classification)
        )

    def forward(self, x):
        x = self.embedding(x)       # Embed input to d_model which is 64
        x = x.unsqueeze(1)          # Add sequence dimension meaning treating our cloumn as a single word sentence
        x = self.transformer(x)     # Pass through Transformer to look at input features
        x = x.mean(dim=1)           # Removing the extra dimension added or also called pooling
        return self.head(x)         # Final output


In [ ]:
model = TabularTransformer(input_dim=X_train.shape[1])
criterion = nn.BCELoss()  # Binary Cross-Entropy loss for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
for epoch in range(10):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.4225
Epoch 2, Loss: 0.3408
Epoch 3, Loss: 0.3930
Epoch 4, Loss: 0.5067
Epoch 5, Loss: 0.3817
Epoch 6, Loss: 0.3907
Epoch 7, Loss: 0.3636
Epoch 8, Loss: 0.5689
Epoch 9, Loss: 0.4359
Epoch 10, Loss: 0.4629


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        all_preds.extend((preds > 0.5).int().squeeze().tolist())
        all_targets.extend(yb.squeeze().tolist())

print("Accuracy:", accuracy_score(all_targets, all_preds))
print("Precision:", precision_score(all_targets, all_preds))
print("Recall:", recall_score(all_targets, all_preds))
print("F1 Score:", f1_score(all_targets, all_preds))


Accuracy: 0.7867803837953091
Precision: 0.635036496350365
Recall: 0.46524064171123
F1 Score: 0.5370370370370371


# 3) Step 3


Comparing Logistic Regression and Transformer Model based on the result

Logistic Regression:
  - Accuracy: 0.79
  - Precision: 0.62
  - Recall: 0.51
  - F1 Score: 0.56

Transformer Model:
  - Accuracy: ~0.78
  - Precision: ~0.63
  - Recall: ~0.46
  - F1 Score: ~0.53

The logistic regression model achieved about the same accuracy but higher recall.


The Transformer model achieved a slightly higher precision and F1 score

# 4) Step 4

I think for predicting churn the logistic regression model is best fit, as recall is high it tells us that it can catch the churner more accurately.


**Recommendation**- Based on the evaluation metrics, Logistic Regression outperformed the Transformer model overall, especially in terms of recall and F1 score, which are critical for churn prediction. While the Transformer model achieved slightly higher precision, the Logistic Regression model was more effective at identifying actual churners, as reflected in its higher recall.